In [1]:
import findspark
findspark.init('/home/yerke/spark-3.3.0-bin-hadoop3')

In [2]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.appName('titanic').getOrCreate()

In [6]:
from pyspark.ml.classification import LogisticRegression

In [7]:
data = spark.read.csv('../course_data/titanic.csv', inferSchema=True, header=True)

In [8]:
data.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [10]:
data.columns

['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [11]:
my_cols = data.select(['Survived',
                       'Pclass',
                       'Sex',
                       'Age',
                       'SibSp',
                       'Parch',
                       'Fare',
                       'Embarked'])

In [12]:
my_final_data = my_cols.na.drop()

In [13]:
from pyspark.ml.feature import (VectorAssembler, VectorIndexer, 
                                OneHotEncoder, StringIndexer)

In [16]:
gender_indexer = StringIndexer(inputCol='Sex', outputCol='SexIndex')
gender_encoder = OneHotEncoder(inputCol='SexIndex', outputCol='SexVec')

In [17]:
embark_indexer = StringIndexer(inputCol='Embarked', outputCol='EmbarkIndex')
embark_encoder = OneHotEncoder(inputCol='SexIndex', outputCol='EmbarkVec')

In [19]:
assembler = VectorAssembler(inputCols=['Pclass', 'SexVec', 'EmbarkVec', 'Age', 'SibSp', 'Parch', 'Fare'],
                            outputCol='features')

In [20]:
from pyspark.ml import Pipeline

In [21]:
log_reg_titanic = LogisticRegression(featuresCol='features', labelCol='Survived')

In [22]:
pipeline = Pipeline(stages=[gender_indexer, embark_indexer,
                            gender_encoder, embark_encoder,
                            assembler, log_reg_titanic])

In [23]:
train_data, test_data = my_final_data.randomSplit([.7,.3])

In [24]:
fit_model= pipeline.fit(train_data)

In [25]:
results = fit_model.transform(test_data)

In [26]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [28]:
my_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='Survived')

In [29]:
results.select('Survived', 'prediction').show()

+--------+----------+
|Survived|prediction|
+--------+----------+
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       1.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
+--------+----------+
only showing top 20 rows



In [30]:
AUC = my_eval.evaluate(results)

In [31]:
AUC

0.807843137254902